# 画像分類

*Computer Vision*コグニティブサービスでは、画像を扱うための便利なモデルがあらかじめ用意されていますが、コンピュータビジョンのために独自のモデルをトレーニングする必要があることがよくあります。例えば、ノースウインド・トレーダーズの小売会社が、レジでカメラで撮影した画像に基づいて、顧客が購入したい食料品を識別する自動レジシステムを作成したいとします。そのためには、画像を分類して購入する商品を識別する分類モデルを訓練する必要があります。

<p style='text-align:center'><img src='./images/image-classification.jpg' alt='A robot holding a clipboard, classifying pictures of an apple, a banana, and an orange'/></p>クリップボードを持ったロボットがリンゴ、バナナ、オレンジの画像を分類しています。

Azureでは、**Custom Vision***コグニティブサービスを使用して、既存の画像に基づいて画像分類モデルをトレーニングすることができます。画像分類ソリューションを作成するには、2つの要素があります。まず、既存の画像を使用して異なるクラスを認識するモデルを訓練する必要があります。次に、モデルが訓練されたら、アプリケーションで利用できるサービスとして公開する必要があります。

## カスタムビジョンリソースを作成する

カスタムビジョンサービスを使用するには、モデルを*トレーニング*するために使用できるAzureリソースと、アプリケーションが使用するために*公開*できるリソースが必要です。どちらか（または両方）のタスクのリソースは、一般的な**Cognitive Services**リソースか、特定の**Custom Vision**リソースにすることができます。これらのタスクのそれぞれに同じコグニティブサービスリソースを使用することもできますし、コストを個別に管理するために、タスクごとに異なるリソース（同じリージョン内）を使用することもできます。

以下の手順を使用して、新しい**Custom Vision**リソースを作成します。

1. 新しいブラウザタブで、[https://portal.azure.com](https://portal.azure.com)のAzureポータルを開き、Azureサブスクリプションに関連付けられたMicrosoftアカウントを使用してサインインします。
2. リソースの作成**ボタンを選択し、*カスタムビジョン**を検索し、以下の設定で**カスタムビジョン**リソースを作成します。:
    - **Create options**: Both
    - **Subscription**: *Your Azure subscription*
    - **Resource group**: *Create a new resource group with a unique name*
    - **Name**: *Enter a unique name*
    - **Training location**: *Choose any available region*
    - **Training pricing tier**: F0
    - **Prediction location**: *The same region as the training resource*
    - **Prediction pricing tier**: F0

    > **注意**: すでにF0カスタムビジョンサービスを契約している場合は、**S0**を選択してください。
    
3. リソースが作成されるのを待ち、2 つのカスタムビジョンリソースがプロビジョニングされていることに注意してください。これらのリソースは、作成したリソースグループに移動して表示することができます。


## Custom Visionプロジェクトを作成する

オブジェクト検出モデルをトレーニングするには、トレーニングリソースに基づいてCustom Visionプロジェクトを作成する必要があります。これを行うには、Custom Vision ポータルを使用します。

1. https://aka.ms/fruit-images からトレーニング画像をダウンロードして抽出します。
2. 別のブラウザタブで、[https://customvision.ai](https://customvision.ai)のCustom Visionポータルを開きます。プロンプトが表示されたら、Azure サブスクリプションに関連付けられた Microsoft アカウントを使用してサインインし、利用規約に同意します。
3. Custom Vision ポータルで、以下の設定で新しいプロジェクトを作成します。:
    - **Name**: Grocery Checkout
    - **Description**: Image classification for groceries
    - **Resource**: *The Custom Vision resource you created previously*
    - **Project Types**: Classification
    - **Classification Types**: Multiclass (single tag per image)
    - **Domains**: Food
4. **\[+\] Add images**をクリックし、先ほど解凍した**apple**フォルダ内のファイルを全て選択します。そして、このように *apple* というタグを指定して画像ファイルをアップロードします。
    <p style='text-align:center'><img src='/images/upload_apples.jpg' alt='Upload apple with apple tag'/></p>
5. 前の手順を繰り返して、**banana**フォルダ内の画像を*banana*タグでアップロードし、**orange**フォルダ内の画像を*orange*タグでアップロードします。
6. Custom Visionプロジェクトでアップロードした画像を見てみましょう。
    <p style='text-align:center'><img src='./images/fruit.jpg' alt='果物のタグ付き画像 - リンゴ15個、バナナ15個、オレンジ15個'/></p>
7. カスタムビジョンプロジェクトで、画像の上にある **トレーニング** をクリックして、タグ付けされた画像を使用して分類モデルをトレーニングします。**クイックトレーニング** オプションを選択し、トレーニングの反復が完了するのを待ちます（1分ほどかかる場合があります）。
8. モデルの反復訓練が完了したら、*Precision*、*Recall*、および*AP*のパフォーマンスメトリクスを確認します - これらは、分類モデルの予測精度を測定するもので、すべてが高いはずです。

## モデルをテストする

Before publishing this iteration of the model for applications to use, you should test it.

1. Above the performance metrics, click **Quick Test**.
2. In the **Image URL** box, type `https://aka.ms/apple-image` and click &#10132;
3. View the predictions returned by your model - the probability score for *apple* should be the highest, like this:
    <p style='text-align:center'><img src='./images/test-apple.jpg' alt='An image with a class prediction of apple'/></p>
4. Close the **Quick Test** window.

## Publish and consume the image classification model

Now you're ready to publish your trained model and use it from a client application.

9. Click **&#128504; Publish** to publish the trained model with the following settings:
    - **Model name**: groceries
    - **Prediction Resource**: *The prediction resource you created previously*.
10. After publishing, click the *settings* (&#9881;) icon at the top right of the **Performance** page to view the project settings. Then, under **General** (on the left), copy the **Project Id** and paste it into the code cell below (replacing **YOUR_PROJECT_ID**).
    <p style='text-align:center'><img src='./images/cv_project_settings.jpg' alt='Project ID in project settings'/></p>

> _**Note**: If you used a **Cognitive Services** resource instead of creating a **Custom Vision** resource at the beginning of this exercise, you can copy its key and endpoint from the right side of the project settings, paste it into the code cell below, and run it to see the results. Otherwise, continue completing the steps below to get the key and endpoint for your Custom Vision prediction resource._

11. At the top left of the **Project Settings** page, click the *Projects Gallery* (&#128065;) icon to return to the Custom Vision portal home page, where your project is now listed.
12. On the Custom Vision portal home page, at the top right, click the *settings* (&#9881;) icon to view the settings for your Custom Vision service. Then, under **Resources**, expand your *prediction* resource (<u>not</u> the training resource) and copy its **Key** and **Endpoint** values to the code cell below, replacing **YOUR_KEY** and **YOUR_ENDPOINT**.
    <p style='text-align:center'><img src='./images/cv_settings.jpg' alt='Prediction resource key and endpoint in custom vision settings'/></p>
13. Run the code cell below to set the variables to your project ID, key, and endpoint values.

In [ ]:
project_id = 'YOUR_PROJECT_ID'
cv_key = 'YOUR_KEY'
cv_endpoint = 'YOUR_ENDPOINT'

model_name = 'groceries' # this must match the model name you set when publishing your model iteration (it's case-sensitive)!
print('Ready to predict using model {} in project {}'.format(model_name, project_id))

Client applications can use the details above to connect to and your custom vision classification model.

Run the following code cell to classifiy a selection of test images using your published model.

> **Note**: Don't worry too much about the details of the code. It uses the Computer Vision SDK for Python to get a class prediction for each image in the /data/image-classification/test-fruit folder

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Get the test images from the data/vision/test folder
test_folder = os.path.join('data', 'image-classification', 'test-fruit')
test_images = os.listdir(test_folder)

# Create an instance of the prediction service
custom_vision_client = CustomVisionPredictionClient(cv_key, endpoint=cv_endpoint)

# Create a figure to display the results
fig = plt.figure(figsize=(16, 8))

# Get the images and show the predicted classes for each one
print('Classifying images in {} ...'.format(test_folder))
for i in range(len(test_images)):
    # Open the image, and use the custom vision model to classify it
    image_contents = open(os.path.join(test_folder, test_images[i]), "rb")
    classification = custom_vision_client.classify_image(project_id, model_name, image_contents.read())
    # The results include a prediction for each tag, in descending order of probability - get the first one
    prediction = classification.predictions[0].tag_name
    # Display the image with its predicted class
    img = Image.open(os.path.join(test_folder, test_images[i]))
    a=fig.add_subplot(len(test_images)/3, 3,i+1)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(prediction)
plt.show()

Hopefully, your image classification model has correctly identified the groceries in the images.

## Learn more

The Custom Vision service offers more capabilities than we've explored in this exercise. For example, you can also use the Custom Vision service to create *object detection* models; which not only classify objects in images, but also identify *bounding boxes* that show the location of the object in the image.

To learn more about the Custom Vision cognitive service, view the [Custom Vision documentation](https://docs.microsoft.com/azure/cognitive-services/custom-vision-service/home)